# ISPA - SearchLight
Script adapted from: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/tree/master/fmri_data

Original paper: https://doi.org/10.1016/j.neuroimage.2019.116205

This script demonstrates how to use the searchlight implementation
available in nilearn to perform group-level decoding using an
inter-subject pattern analysis (ISPA) scheme.

## Importing packages

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op
import re
import glob
import tarfile
import urllib

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression

from nilearn.image import new_img_like, concat_imgs, clean_img
from nilearn.decoding import SearchLight

import nibabel as nb
from nilearn import plotting

import time  # Import the time module


## Opening files to work with

In [ ]:
# Set the path to your GLM directory
data_dir = '/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/PROC_DATA/derivatives/GLM'

# getting a full list of available subjects
files = os.listdir(data_dir)
pattern = r'sub-(\d+)_LSA'
subject_numbers = [re.search(pattern, file_name).group(1) for file_name in files if re.search(pattern, file_name)]

# List of subjects to include, empty means include all
subjects_to_include = []  # Fill with subject identifiers or leave empty to include all

beta_flist = []
y = []
subj_vect = []
runs = []
        
# Create lists from beta map filenames in 'beta_flist', labels in 'y', and subject numbers in 'subj_vect'.
for subject_number in subject_numbers:
    # If subjects_to_include is empty or subject_number is in the list, process the subject
    if not subjects_to_include or subject_number in subjects_to_include:
        subject_directory = os.path.join(data_dir, f'sub-{subject_number}_LSA')

        # List beta map files for the current subject
        beta_map_files = glob.glob(os.path.join(subject_directory, f'sub-{subject_number}_run*_*.nii.gz'))
        beta_map_files.sort()
        beta_flist.extend(beta_map_files)

        # Extract the label from the beta map filename
        for beta_map_file in beta_map_files:
            label = beta_map_file.split('_')[-3]  # Assumes label is the third last part of the filename
            if "InfOther" in label:  # Adjust labels to consider Own vs. Other (irrelevant to valence)
                label = "Other"
            elif "InfOwn" in label:
                label = "Own"
            y.append(label)
            subj_vect.append(subject_number)

        # Extract run info from the beta map filename (to be used in standardization)
        for beta_map_file in beta_map_files:
            run = beta_map_file.split('_')[-4]  # Assumes run is the fourth last part of the filename
            runs.append(run)

## STANDARDIZATIOIN (per run and subject)

# read image data
print("Reading beta maps from all the subjects...")

fmri_nii_dict = {}

for beta_path in beta_flist:
    parts = re.split(r'[/_]', beta_path)
    subj = parts[-6]
    run = 'run-'+parts[-4]
    condition = parts[-3]
    beta = parts[-2]
    
    image = nb.load(beta_path)
    
    # Check if the subject already exists in the dictionary
    if subj in fmri_nii_dict:
        # If the run exists for the subject, append the data to the existing list
        if run in fmri_nii_dict[subj]:
            fmri_nii_dict[subj][run].append({'condition': condition, 'beta': beta, 'image': image})
        # If the run doesn't exist, create a new entry for the run
        else:
            fmri_nii_dict[subj][run] = [{'condition': condition,'beta': beta, 'image': image}]
    # If the subject doesn't exist, create a new entry for the subject and run
    else:
        fmri_nii_dict[subj] = {run: [{'condition': condition,'beta': beta, 'image': image}]}

# Printing the created dictionary
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        print(f"Subject: {subj}, Run: {run}")
        for item in run_value:
            print(item)
        print('\n')
        
## CONCATENATE BETAS IMAGES ##

concat_imgs_dict = {}
concat_imgs_stand_dict = {}

# Concatenating the images for the same run and subject
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        images_to_concat = [item['image'] for item in run_value]
        concatenated_image = concat_imgs(images_to_concat)
        print(f"Subject: {subj}, Run: {run}, Concatenated Beta Images Shape: {concatenated_image.shape}")
        
        # Adding concatenated image to the concat_imgs_dict
        if subj in concat_imgs_dict:
            concat_imgs_dict[subj][run] = concatenated_image
        else:
            concat_imgs_dict[subj] = {run: concatenated_image}

        # Standardized beta maps with clean_img()
        stand_image = clean_img(concatenated_image, standardize=True, detrend=False)
        print(f"Subject: {subj}, Run: {run}, Concatenated Standardized Beta Images Shape: {stand_image.shape}")
        
        # Adding standardized image to the concat_imgs_stand_dict
        if subj in concat_imgs_stand_dict:
            concat_imgs_stand_dict[subj][run] = stand_image
        else:
            concat_imgs_stand_dict[subj] = {run: stand_image}
            
# Concatenate all subjects #

all_images_to_concat = []

for subj, subj_value in concat_imgs_stand_dict.items():
    for run, stand_image in subj_value.items():
        all_images_to_concat.append(stand_image)

all_betas_stand = concat_imgs(all_images_to_concat) 

print(f"All Betas Stand Shape: {all_betas_stand.shape}")

# Adjust Masks 

In [ ]:
from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img

# reading brain mask
mask_brain = nb.load("/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/brain_mask.nii.gz")

# Check whether ROI and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])
if not beta_img.shape == mask_brain.shape:
    resampled_mask_brain = resample_to_img(mask_brain, beta_img)
print("Shape of original Beta image: %s" % (beta_img.shape,))
print("Shape of original ROI image: %s" % (mask_brain.shape,))
print("Shape of resampled ROI image: %s" % (resampled_mask_brain.shape,))

In [ ]:
from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img
roi_mask_nii = nb.load("/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/BIDS_Nilearn/2ndLvPilotSPACE_Binar_NAccBrian_OFC_Septal.nii")

# Check whether ROI and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])
if not beta_img.shape == roi_mask_nii.shape:
    resampled_ROI = resample_to_img(roi_mask_nii, beta_img)
print("Shape of original Beta image: %s" % (beta_img.shape,))
print("Shape of original ROI image: %s" % (roi_mask_nii.shape,))
print("Shape of resampled ROI image: %s" % (resampled_ROI.shape,))

# Setting up for leave-one-subject-out cross-validation 

In [ ]:
# set up leave-one-subject-out cross-validation
loso = LeaveOneGroupOut()
unique_subjects = np.unique(subj_vect)
n_splits = loso.get_n_splits(groups=unique_subjects)

In [ ]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []
print("Launching cross-validation...")
for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    single_split = [(train_inds,test_inds)]
    y_train = y[train_inds]
    n_samples = len(y_train)
    class_labels = np.unique(y_train)
    clf = LogisticRegression()
    searchlight = SearchLight(resampled_mask_brain,
                              #process_mask_img=resampled_ROI,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=1,
                              cv=single_split,
                              estimator=clf)
    print("...mapping the data (this takes a long time) and fitting the model in each sphere")
    searchlight.fit(all_betas_stand, y)

    single_split_nii = new_img_like(resampled_mask_brain,searchlight.scores_ - chance_level)
    single_split_path = op.join(output_dir, 'ispa_searchlight_accuracy_split{:02d}of{:02d}.nii'.format(split_ind+1,n_splits))
    print('Saving score map for cross-validation fold number {:02d}'.format(split_ind+1))
    single_split_nii.to_filename(single_split_path)
    single_split_path_list.append(single_split_path)

# Trying to do Non-parametric Permutations with nilearn
Originally it was performed with SnPM MATLAB toolbox to analyse the single-fold (for the inter-subject cross-validation of ISPA) accuracy maps, with 1000 permutations and a significance threshold (p<0,05, FWE corrected).
Here, we are trying to do the same, but using nilearn.

Original SnPM batch: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/blob/master/fmri_data/snpm_batch.m

## Here we have tried to implement the permutatoin scheme suggested by Stelzer et al. 2013, by permuting labels for X searchlights per subject

In [ ]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []

# Number of permutations within subject
n_permutations = 100

# List to hold the timings
timings = []

print("Launching cross-validation...")
total_start_time = time.time()  # Capture the start time of the entire process

for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    split_start_time = time.time()  # Start time for this split
    
    for permutation in range(n_permutations):
        print(f"...within-subject permutation {permutation+1:02d} of {n_permutations}")
        permutation_start_time = time.time()  # Start time for this permutation
        
        # Append the results to timing list
        #timings.append({f"Start time for split {split_ind+1}': split_start_time})
        
        # Shuffle y labels for this permutation
        np.random.seed(permutation)  # Setting the seed for reproducibility
        y_shuffled = np.random.permutation(y)

        # Set up the classifier and searchlight with the shuffled labels
        clf = LogisticRegression()
        searchlight = SearchLight(resampled_mask_brain,
                                  radius=searchlight_radius,
                                  n_jobs=n_jobs,
                                  verbose=1,
                                  cv=[(train_inds, test_inds)],
                                  estimator=clf)
        
        searchlight_start_time = time.time()  # Start time for this searchlight
        # Fit the model with shuffled labels
        searchlight.fit(all_betas_stand, y_shuffled)

        # Compute accuracy map and subtract chance level
        accuracy_map = searchlight.scores_ - chance_level

        # Save the accuracy map for this permutation and split
        single_split_nii = new_img_like(resampled_mask_brain, accuracy_map)
        output_filename = f'split_{split_ind+1:02d}_permutation_{permutation+1:03d}_accuracy.nii'
        single_split_path = os.path.join(output_dir, output_filename)
        print(f'Saving permutation score map {permutation+1:03d} for split {split_ind+1:02d}')
        single_split_nii.to_filename(single_split_path)
        
        # Print duration for this search light
        searchlight_end_time = time.time()
    
        print(f"Searchlight {permutation+1} completed in {searchlight_end_time - searchlight_start_time:.2f} seconds.")
        # Append the results to timing list
        timings.append({f"Searchlight {permutation+1}: {searchlight_end_time - searchlight_start_time:.2f}"})
        
    # Print duration for this split
    split_end_time = time.time()
    print(f"Split {split_ind+1} completed in {split_end_time - split_start_time:.2f} seconds.")
    timings.append({f"Split {split_ind+1}: {split_end_time - split_start_time:.2f}"})

# Print the total duration
total_end_time = time.time()
print(f"Total processing time: {total_end_time - total_start_time:.2f} seconds.")
timings.append({f"Total processing time: {total_end_time - total_start_time:.2f}"})

new_df = pd.DataFrame(timings)
new_df.to_csv('timings_all.csv', index=False)

## Randomly select one accuracy map per subject and average (repeat 10n5 x).

## Pool of 10n5 chance group accuracy maps


In [102]:
           
import numpy as np
import nibabel as nib
import os
import glob
from nilearn.image import new_img_like

# Define input and output directories
input_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut"

os.makedirs(output_dir, exist_ok=True)

# Function to extract unique split identifiers based on existing files in directory
def get_unique_identifiers(directory, pattern):
    files = glob.glob(os.path.join(directory, pattern))
    unique_ids = set()
    for file in files:
        match = re.search(r"split_(\d+)_permutation_(\d+)_accuracy\.nii", file)
        if match:
            unique_ids.add(match.group(1))
    return list(unique_ids)

# Use the function to get a list of unique split identifiers
split_identifiers = get_unique_identifiers(input_dir, "split_*_permutation_*_accuracy.nii")

# fix identifier for testing
#split_identifiers= ['01','02','03']

# Preload maps paths for each split identifier
split_maps_paths = {}
for split_identifier in split_identifiers:
    pattern = os.path.join(input_dir, f"split_{split_identifier}_permutation_*_accuracy.nii")
    split_maps_paths[split_identifier] = glob.glob(pattern)


In [106]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import random
import csv

n_iterations = 100000
output_dir = os.path.join(input_dir, "permu_group_acc_maps")
timings_file = os.path.join(output_dir, "timings.csv")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load an example NIfTI image to get the shape and affine
example_img_path = split_maps_paths[split_identifiers[0]][0]
example_img = nib.load(example_img_path)
img_shape = example_img.shape
affine = example_img.affine

def perform_iteration(iteration, output_dir):
    start_time = time.time()  # Start timing the iteration
    #print(f"Starting iteration {iteration+1}")  # Debugging line
    sum_of_maps = np.zeros(img_shape)
    selected_maps = []  # Initialize a list to store the paths of selected maps
    
    # Randomly select one map per subject/split and accumulate
    for split_id, map_paths in split_maps_paths.items():
        selected_map_path = random.choice(map_paths)
        selected_maps.append(selected_map_path)  # Store the selected map path
        #print(f"Selected map for {split_id}: {selected_map_path}")  # Debugging line
        map_data = nib.load(selected_map_path).get_fdata()
        sum_of_maps += map_data
    
    averaged_map_data = sum_of_maps / len(split_maps_paths)
    averaged_map = nib.Nifti1Image(averaged_map_data, affine)
    averaged_map_filename = f"permuted_group_accuracy_map_{iteration+1:03d}.nii"
    averaged_map_path = os.path.join(output_dir, averaged_map_filename)
    averaged_map.to_filename(averaged_map_path)
    
    # Finish the iteration
    end_time = time.time()
    duration = end_time - start_time
    return {"Iteration": iteration + 1, "Duration (s)": duration}
    
    #print(f"Saved permuted group accuracy map for iteration {iteration+1}")
    
# Measure the total processing time
total_start_time = time.time()

# Use ProcessPoolExecutor to run iterations in parallel
iteration_timings = []
with ProcessPoolExecutor() as executor:
    # Start all iterations and return futures
    futures = [executor.submit(perform_iteration, iteration, output_dir) for iteration in range(n_iterations)]
    
    # As each future completes, record its duration
    for future in as_completed(futures):
        iteration_timings.append(future.result())

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total processing time: {total_duration:.2f} seconds.")

# Add total processing time to the timings list
iteration_timings.append({"Iteration": "Total", "Duration (s)": total_duration, "Selected Maps": selected_maps})

# Save timings to a CSV file
with open(timings_file, 'w', newline='') as file:
    fieldnames = ["Iteration", "Duration (s)"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(iteration_timings)
    
# Save the collected data to a CSV file
csv_file_path = os.path.join(output_dir, "selected_maps.csv")
with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Iteration', 'Duration (s)', 'Selected Maps']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in iteration_timings:  # Use iteration_timings here
        # Convert the list of selected maps to a string to save in the CSV
        result['Selected Maps'] = '|'.join(result['Selected Maps'])
        writer.writerow(result)

print(f"Timings saved to {timings_file}")


Total processing time: 1541.19 seconds.


NameError: name 'iteration_results' is not defined

## Need to figure it out how to do steps C and D in the Stelzer paper


In [ ]:
import numpy as np
import nibabel as nib
import os
from concurrent.futures import ProcessPoolExecutor

input_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut/permu_group_acc_maps"

# Setup variables
n_permutations = 100000
voxel_shape = example_img.shape[:3]  # Assuming example_img is defined
n_bins = 100
histogram_range = (0, 1)  # Assuming accuracies are between 0 and 1

# Initialize histograms for each voxel
voxel_histograms = np.zeros(voxel_shape + (n_bins,))

# Function to update histograms given the accuracies of one permutation
def update_histograms(permutation_accuracies, histograms, bins, range):
    hist, _ = np.histogram(permutation_accuracies, bins=bins, range=range)
    histograms += hist

# Assuming you have a function load_permutation_accuracies(index) that loads the accuracies for a given permutation
def process_permutation(index):
    permutation_accuracies = load_permutation_accuracies(index)  # This function needs to be defined
    update_histograms(permutation_accuracies, voxel_histograms, n_bins, histogram_range)

# Process each permutation using parallel processing
with ProcessPoolExecutor() as executor:
    list(executor.map(process_permutation, range(n_permutations)))

# Now, voxel_histograms contains the updated histograms for each voxel across all permutations

# Calculate the cumulative distribution function (CDF) from histograms
voxel_cdfs = np.cumsum(voxel_histograms, axis=3) / n_permutations

# Find threshold accuracy where the right-tailed area of the CDF is below 0.001
threshold_indices = np.searchsorted(voxel_cdfs, 0.999, side='right')
bin_edges = np.linspace(histogram_range[0], histogram_range[1], num=n_bins+1)
threshold_accuracies = bin_edges[threshold_indices]

# Save threshold map
threshold_map = nib.Nifti1Image(threshold_accuracies, affine=example_img.affine)
nib.save(threshold_map, os.path.join(output_dir, 'threshold_accuracy_map.nii'))


# Generating figures of sinificant brain regions from SnPM


In [ ]:
result_dir = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level'
fname = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/lP_FDR+.img'

#brain_nii = nb.load(result_dir + '/lP_FWE+.img')
img = nb.load(fname)
nb.save(img, fname.replace('.img', '.nii'))
display = plotting.plot_glass_brain(None, display_mode='lyrz')
color = 'r'
display.add_contours(img, filled=True, levels=[1], colors=color)
display.title('regions uncovered by ISPA, cluster-level FWE')
#display.savefig(result_dir + '/{}_{}.png'.format(dataset_name, decoding_type))

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-9, 0, 5, 10),
                      title='SnPM FWE cluster mass', vmax=vmax, display_mode='x', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-20,-15,-9,5),
                     vmax=vmax, display_mode='z', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)